# e-greedy Monte Carlo Control
An implementation of MC control through "epsilon greedy"

The gridworld has the shape(3,4) with a winning state "w"(0,3), and a lossing state "l"(1,3), a non valid state "x"(2,1) and a start state s(3,0)

|  |  |  |  |
|---|---|---|---|
|  |  |  | w |
|  |  |  | l |
|  | x |  |  |
| s |  |  |  |

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

import grid_world

### Disccount factor

In [2]:

GAMMA = 0.9

### Auxiliary function to display the values of a policy after finishing iterative policy evaluation

In [3]:
def print_values(V,grid):
    for i in range(grid.width):
        print("--------------------------")
        for j in range(grid.height):
            v = V.get((i,j),0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")
        print("")

### Auxiliary function to display a stochastic policy

In [4]:
def print_policy(P,grid):
    for i in range(grid.width):
        print("---------------------------")
        for j in range(grid.height):
            a = P.get((i,j),' ')
            if isinstance(a,dict):
                a = list(a)[0]
            print("  %s  |" % a, end="")
        print("")

### From or defined grid world file, import a negative grid ,retrieve all actions and states and print grid rewards
Negative grid is used to encourage the agent to find a shortest path to the goal

In [5]:
grid = grid_world.Grid.standard_grid()
states = grid.all_states()
actions = list(set([action   for action_tup in grid.actions.values() for action in action_tup]))

In [6]:
def argmax_dict(dictionary):
    # returns the argmax key and the max value from a dictionary
    # will be used for policy improvement from Q
    max_key = None
    max_val = float("-inf")
    
    for k,v in dictionary.items():
        if v > max_val:
            max_val = v
            max_key = k
            
    return max_key,max_val
        
argmax_dict({"a":1,"b":2})

('b', 2)

In [7]:
actions

['L', 'R', 'U', 'D']

In [8]:
def epsilon_greedy_action(policy,state,epsilon=0.1):
    # choose an action using epsilon-greedy strategy
    probability = np.random.random()
    greedy_action = policy[s]
    non_greedy_actions = list(set(actions)-set(greedy_action))
    
    if probability < epsilon:
        return np.random.choice(non_greedy_actions)
    else: 
        return policy[state]

In [9]:
def play_game(grid,policy,gamma=0.9,epsilon  = 0.1):
    # remove exploring starts
    #start_states = list(grid.actions.keys())
    #start_index = np.random.choice(len(start_states))
    #grid.set_state(start_states[start_index])
    
    #s = grid.current_state()
    #a = np.random.choice(actions)
    
    s = (2,0)
    grid.set_state(s)
    a = epsilon_greedy_action(policy,s,epsilon)
    states_actions_and_rewards = [(s,a,0)] 
    #print(s,a,grid.game_over())
    # play a game until game is finished and record all new states and rewards
    while not grid.game_over():
        
        old_s = grid.current_state()
        r = grid.move(a)
        s = grid.current_state()
        
        # if agent bumps into a wall, the episode ends with negative reward 
        # to make it avoid this state,action in the future
        if old_s == s :
            states_actions_and_rewards.append((s,None,-1))
            break
        elif grid.game_over():
            states_actions_and_rewards.append((s,None,r))
            break
        else:
            a = epsilon_greedy_action(policy,s,epsilon)
            states_actions_and_rewards.append((s,a,r))
            
        
    # calculate every state returns by going backwards
    states_and_actions_returns = []
    
    G=0
    for s,a,r in reversed(states_actions_and_rewards):
        
        G = r +gamma*G
        states_and_actions_returns.append((s,a,G))
        
    return states_and_actions_returns


In [10]:
grid = grid_world.Grid.standard_grid()

In [11]:
print("Rewards of grid")
print_values(grid.rewards,grid)

Rewards of grid
--------------------------
 0.00| 0.00| 0.00| 1.00|
--------------------------
 0.00| 0.00| 0.00|-1.00|
--------------------------
 0.00| 0.00| 0.00| 0.00|


### Initialize random policy and Q(s,a)

In [12]:
policy = {}

for s in grid.actions.keys():
    policy[s] = np.random.choice(actions)
print_policy(policy,grid)

---------------------------
  R  |  D  |  L  |     |
---------------------------
  D  |     |  D  |     |
---------------------------
  R  |  R  |  U  |  D  |


In [13]:
Q = defaultdict(lambda: defaultdict(lambda:0))
Q[(0,0)][0]

0

In [14]:
def MS_epsilon_greedy(grid,policy,episodes,gamma,epsilon=0.1):
    Q = defaultdict(lambda: defaultdict(lambda:0))
    state_action_visit_count = defaultdict(lambda: defaultdict(lambda:1))
    
    for i in range(episodes):
        
        states_actions_and_returns = play_game(grid,policy,gamma,epsilon)
        visited_states_actions = set()
        
        #  policy evaluation
        for s,a,G in states_actions_and_returns:
            
            if (s,a) not in visited_states_actions:
                
                # incremental average of return calculation
                Q[s][a] = Q[s][a] + (1.0/state_action_visit_count[s][a])*(G-Q[s][a])
                visited_states_actions.add((s,a))
                state_action_visit_count[s][a] += 1

        
            
                
            #policy improvement
        for s in states:
            state_argmax = argmax_dict(Q[s])
            if not state_argmax[0] is None:
                policy[s] = state_argmax[0]
    return policy,Q
        
policy,Q = MS_epsilon_greedy(grid,policy,50000,GAMMA,0.2)

In [15]:
policy

{(0, 0): 'D',
 (0, 1): 'R',
 (0, 2): 'R',
 (1, 0): 'D',
 (1, 2): 'U',
 (2, 0): 'R',
 (2, 1): 'R',
 (2, 2): 'U',
 (2, 3): 'L'}

In [16]:
print("Policy")
print_policy(policy,grid)

Policy
---------------------------
  D  |  R  |  R  |     |
---------------------------
  D  |     |  U  |     |
---------------------------
  R  |  R  |  U  |  L  |


In [17]:
for state in policy.keys():
    print(state," ",policy[state])
    for action in actions:
        print("----",action," ",Q[state][action])

(0, 0)   D
---- L   0
---- R   -0.35912999999999995
---- U   -0.9
---- D   -0.10594330755049175
(0, 1)   R
---- L   0
---- R   0.7907625
---- U   -0.9
---- D   -0.9
(0, 2)   R
---- L   -0.81
---- R   0.9
---- U   -0.9
---- D   0.5069663088510635
(1, 0)   D
---- L   -0.9
---- R   -0.9
---- U   -0.18491139892854802
---- D   0.05128037272957729
(1, 2)   U
---- L   -0.9
---- R   -0.9
---- U   0.6871141270453736
---- D   0.3147254589101253
(2, 0)   R
---- L   -0.9
---- R   0.153615286086542
---- U   -0.01782825415930647
---- D   -0.9
(2, 1)   R
---- L   -0.6368156355323694
---- R   0.33260242252349187
---- U   -0.9
---- D   -0.9
(2, 2)   U
---- L   -0.6744037717832772
---- R   0.06783893954605508
---- U   0.5011661642633631
---- D   -0.9
(2, 3)   L
---- L   0.3444384474349818
---- R   -0.9
---- U   -0.9
---- D   -0.9


## Conclusions
* No infinite loops were created during training(as in exploring starts) because at some point exploration breaks them, but the final policy contains several :( 
* Same as all monte carlo, no model of env dynamics are needed
* No need for exploring starts is useful in cases were the initial state is not something under our control